In [71]:
import ctypes
import numpy as np
import random
from scipy import sparse

'''# speed comparison of four approaches    '''
'''    # native python libraries                        '''
'''    # specialist libraries                                   '''
'''    # ctype method calls into C                                      '''
'''    # wholesale modular call into C from compiled library                    '''
'''                                                                                 '''
'''# further optimisations maybe possible with the four approaches                      '''
'''# note that mutate request internally casts the numpy population into a list of lists    '''

class MutateRequest:
    def __init__(self, population):
        number_of_mutations = get_number_of_mutations()
        self.population     = population
        self.population     = [list(algo) for algo in population]
        self.rows_algos     = len(population)
        self.columns_genes  = len(population[0])
        self.number_of_mutations = number_of_mutations

def get_number_of_mutations():
    number_of_mutations = 3
    return number_of_mutations

def build_population():
    low         = -2
    high        =  2
    cohort_size = (8, 6) # columns_genes , rows_algos
    population  = np.random.uniform(low, high, size=cohort_size)
    
    return population

def base_mutate(mutate_request):
    ''' does not use numpy; only uses naive base library - math, itertools, random etc are allowed'''
    population          = mutate_request.population
    number_of_mutations = mutate_request.number_of_mutations
    rows_algos          = mutate_request.rows_algos
    columns_genes       = mutate_request.columns_genes
    base_mutants = population
    
    # arbitrary loop nesting order here - this is best for 
    for mutation in range(number_of_mutations):
        for algo in base_mutants:
            random_place    = random.randrange(columns_genes)
            starting_gene   = algo[random_place]
            a_float         = random.random()
            algo.pop(random_place)
            mutant_gene     = starting_gene + a_float
            algo.insert(random_place, mutant_gene)
    return base_mutants

def numpy_mutate(mutate_request):
    population          = mutate_request.population
    population          = np.asarray(population)
    
    number_of_mutations = mutate_request.number_of_mutations
    all_loci            = mutate_request.rows_algos * mutate_request.columns_genes
    
    mutation_rate       = number_of_mutations/all_loci
    
    mutation_matrix = scipy.sparse.random(population.shape[0], population.shape[1], density=mutation_rate)
    pre_mutants     = np.copy(population)
    mutants         = np.add(pre_mutants, mutation_matrix.todense())
    
    return mutants

if __name__ == '__main__':
    
    original_population = build_population()
    mutate_request      = MutateRequest(population)
    base_mutants        = base_mutate(mutate_request)
    numpy_mutants       = numpy_mutate(mutate_request)
    

In [75]:
def test_base_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = base_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      list),  'Error: should be a list'
    assert isinstance(mutants[0],   list),  'Error: should be a list'
    assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    
def test_numpy_mutate():
    number_of_mutations   = get_number_of_mutations()
    population            = build_population()
    mutate_request        = MutateRequest(population)
    mutants               = numpy_mutate(mutate_request)
    max_parity_sum        = population.shape[0] * population.shape[1]
    rows_algos            = mutate_request.rows_algos
    
    #form
    assert isinstance(mutants,      np.ndarray),  'Error: should be an np array'
    assert isinstance(mutants[0],   np.ndarray),  'Error: should be an np array'
    print('\nmutants[0][0]\n', mutants[0][0], type(mutants[0][0]))
    #assert isinstance(mutants[0][0],float), 'Error: should be a float'
    
    #content
    flat_population = list(np.ravel(np.asarray(population)))
    flat_mutants    = list(np.ravel(np.asarray(mutants)))
    both            = zip(flat_population, flat_mutants)
    parity_check    = sum([int(x[0]==x[1]) for x in both])
    if number_of_mutations > 0:
        assert parity_check <= max_parity_sum-rows_algos, 'Error: there should be at least one difference per algo'
    assert parity_check > 0, 'Error: they should not be completely different'
    print('pass')
    

def test_build_population():
    population = build_population()
    assert isinstance(population,       np.ndarray),    'Error: should be array of arrays'
    assert isinstance(population[0],    np.ndarray),    'Error: should be an array'
    assert isinstance(population[0][0], np.float64),    'Error: should be a float'
    print('pass')

def tests():
    test_base_mutate()
    test_build_population()
    test_numpy_mutate()
    
tests()

pass
pass

mutants[0][0]
 [[ 1.37540434  0.24865622  1.59741303 -0.03519354  1.16889392  0.98668862]] <class 'numpy.matrix'>


AssertionError: Error: there should be at least one difference per algo

In [63]:
'''
# Build mutate_interface
'''

'\n# Build mutate_interface\n'

In [15]:
'''# Note

 class ctypes.Array(*args)

    Abstract base class for arrays.

    The recommended way to create concrete array types is by multiplying any ctypes data type
    with a positive integer. Alternatively, you can subclass this type and define _length_ 
    and _type_ class variables. Array elements can be read and written using standard subscript 
    and slice accesses; for slice reads, the resulting object is not itself an Array.

'''

'# Note\n\n class ctypes.Array(*args)\n\n    Abstract base class for arrays.\n\n    The recommended way to create concrete array types is by multiplying any ctypes data type\n    with a positive integer. Alternatively, you can subclass this type and define _length_ \n    and _type_ class variables. Array elements can be read and written using standard subscript \n    and slice accesses; for slice reads, the resulting object is not itself an Array.\n\n'

In [90]:
cdll.LoadLibrary("libc.so.6")

<CDLL 'libc.so.6', handle 7faa37933500 at 0x7faa30a70ed0>

In [91]:
libc = CDLL("libc.so.6")

In [92]:
c_wchar_p("Hello, World")

c_wchar_p(140368937449840)

In [93]:
printf = libc.printf

In [94]:
printf()

6

In [95]:
printf("Hello World")

1

In [96]:
s = "Hello, World"

In [97]:
c_s = c_wchar_p(s)

In [98]:
printf(b"Hello, %s\n", b"World!")

14

In [101]:
print(c_s.value)

Hello, World


In [102]:
print(c_s)

c_wchar_p(140368937837360)


In [103]:
print(s)                # first object is unchanged

Hello, World


In [104]:
printf(b"An int %d, a double %f\n", 1234, c_double(3.14))

31

In [105]:
strchr = libc.strchr

In [106]:
strchr(b"abcdef", ord("d"))  

816650675

In [107]:
i = c_int()

In [108]:
f = c_float()

In [109]:
s = create_string_buffer(b'\000' * 32)

In [110]:
print(i.value, f.value, repr(s.value))

0 0.0 b''


In [111]:
libc.sscanf(b"1 3.14 Hello", b"%d %f %s", byref(i), byref(f), s)

3

In [112]:
print(i.value, f.value, repr(s.value))

1 3.140000104904175 b'Hello'


In [123]:
class POINT(Structure):
    _fields_ = [("x", c_int),
                ("y", c_int)]

In [124]:
point = POINT(10, 20)

In [125]:
print(point.x, point.y)

10 20


In [126]:
point = POINT(y=5)

In [127]:
print(point.x, point.y)

0 5


In [128]:
POINT(1, 2)

In [129]:
POINT(-1, 2)

In [130]:
#POINT(1, 2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-120-a3808173cd67> in <module>
----> 1 POINT(1, 2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-120-a3808173cd67> in <module>\n----> 1 POINT(1, 2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [131]:
# POINT(1, -2.0001)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-121-bfc065603ff4> in <module>
----> 1 POINT(1, -2.0001)

TypeError: int expected instead of float


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-121-bfc065603ff4> in <module>\n----> 1 POINT(1, -2.0001)\n\nTypeError: int expected instead of float\n\n\n'

In [132]:
POINT(1, 2)

In [133]:
class RECT(Structure):
    _fields_ = [("upperleft", POINT),
                ("lowerright", POINT)]

In [134]:
rc = RECT(point)

In [135]:
print(rc.upperleft.x, rc.upperleft.y)

0 5


In [136]:
print(rc.lowerright.x, rc.lowerright.y)

0 0


In [137]:
TenPointsArrayType = POINT * 10

In [138]:
r = RECT(POINT(1, 2), POINT(3, 4))

In [139]:
r = RECT((1, 2), (3, 4))

In [140]:
print(POINT.x)

<Field type=c_int, ofs=0, size=4>


In [141]:
print(POINT.y)

<Field type=c_int, ofs=4, size=4>


In [142]:
class Int(Structure):
    _fields_ = [("first_16", c_int, 16),
                ("second_16", c_int, 16)]

In [143]:
print(Int.first_16)

<Field type=c_int, ofs=0:0, bits=16>


In [144]:
print(Int.second_16)

<Field type=c_int, ofs=0:16, bits=16>


In [145]:
class POINT(Structure):
    _fields_ = ("x", c_int), ("y", c_int)

In [153]:
class MyStruct(Structure):
    _fields_ = [("a", c_int),
                ("b", c_float),
                ("point_array", POINT * 4)]

In [154]:
print(len(MyStruct().point_array))

4


In [155]:
arr = TenPointsArrayType()

In [156]:
for pt in arr:
    print(pt.x, pt.y)

0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0


In [157]:
TenIntegers = c_int * 10

In [158]:
ii = TenIntegers(1, 2, 3, 4, 5, 6, 7, 8, 9, 10)

In [159]:
print(ii)

In [160]:
for i in ii: print(i, end=" ")

1 2 3 4 5 6 7 8 9 10 

In [161]:
i = c_int(42)

In [162]:
pi = pointer(i)

In [163]:
pi.contents

c_int(42)

In [164]:
pi.contents is i

False

In [165]:
pi.contents is pi.contents

False

In [166]:
i = c_int(99)

In [167]:
pi.contents = i

In [168]:
pi.contents

c_int(99)

In [169]:
pi[0]

99

In [170]:
print(i)

c_int(99)


In [171]:
pi[0] = 22

In [172]:
print(i)

c_int(22)


In [173]:
PI = POINTER(c_int)

In [174]:
PI

__main__.LP_c_int

In [175]:
# PI(42)

'''
---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<ipython-input-52-622c40ff30d9> in <module>
----> 1 PI(42)

TypeError: expected c_int instead of int


'''

'\n---------------------------------------------------------------------------\nTypeError                                 Traceback (most recent call last)\n<ipython-input-52-622c40ff30d9> in <module>\n----> 1 PI(42)\n\nTypeError: expected c_int instead of int\n\n\n'

In [176]:
PI(c_int(42))

In [177]:
null_ptr = POINTER(c_int)()

In [178]:
print(bool(null_ptr))

False


In [179]:
class Bar(Structure):
    _fields_ = [("count", c_int), ("values", POINTER(c_int))]

In [180]:
bar = Bar()

In [181]:
bar.values = (c_int * 3)(1, 2, 3)

In [182]:
bar.count = 3

In [183]:
for i in range(bar.count):
    print(bar.values[i])

1
2
3


In [184]:
IntArray5 = c_int * 5

In [185]:
ia = IntArray5(5, 1, 7, 33, 99)

In [186]:
qsort = libc.qsort

In [187]:
qsort.restype = None

In [188]:
CMPFUNC = CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))



In [193]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return 0

In [194]:
cmp_func = CMPFUNC(py_cmp_func)

In [195]:
qsort(ia, len(ia), sizeof(c_int), cmp_func) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 5 7
py_cmp_func 1 7


In [196]:
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [197]:
qsort(ia, len(ia), sizeof(c_int), CMPFUNC(py_cmp_func)) 

py_cmp_func 5 1
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [202]:
for i in ia: print(i, end=" ")




1 5 7 33 99 

In [203]:
@CFUNCTYPE(c_int, POINTER(c_int), POINTER(c_int))
def py_cmp_func(a, b):
    print("py_cmp_func", a[0], b[0])
    return a[0] - b[0]

In [204]:
qsort(ia, len(ia), sizeof(c_int), py_cmp_func)

py_cmp_func 1 5
py_cmp_func 33 99
py_cmp_func 7 33
py_cmp_func 1 7
py_cmp_func 5 7


In [205]:
short_array = (c_short * 4)()

In [206]:
print(sizeof(short_array))

8


In [208]:
#resize(short_array, 4)

'''
---------------------------------------------------------------------------
ValueError                                Traceback (most recent call last)
<ipython-input-207-5d3303a611a5> in <module>
----> 1 resize(short_array, 4)
      2 
      3 

ValueError: minimum size is 8

'''

'\n---------------------------------------------------------------------------\nValueError                                Traceback (most recent call last)\n<ipython-input-207-5d3303a611a5> in <module>\n----> 1 resize(short_array, 4)\n      2 \n      3 \n\nValueError: minimum size is 8\n\n'

In [209]:
resize(short_array, 32)

In [210]:
sizeof(short_array)

32

In [211]:
sizeof(type(short_array))

8